### Raw Data Download Notebook

*Important Note:* This notebook runs using a different environment than this project's computational notebook. ObsPy is required, along with numpy and matplotlib

Zali et al. 2024 pulls horizontal seismic data from a single horizontal component from a single station, demeaned and detrended but not converted from instrument response. The data is pulled from March 12 to June 24, ~104 days, beginning 7 days before the Geldingadalir eruption began. The data is also downsampled to the minimum sampling rate necessary to observe the local volcanic tremor.

------------------

This notebook will download 100 days of data, beginning on March 1st, 2005 and ending June 9th, 2005, from sensor CC.SEP of the Cascade Chain Volcano Monitoring network, located on Mount St. Helens. Mount St. Helens had a series of eruptions of ash and steam between 2004 and 2008, but this will focus on the second explosive eruption on March 8th, 2005 that was preceeded by seismic activity. https://www.usgs.gov/volcanoes/mount-st.-helens/science/2004-2008-renewed-volcanic-activity

In [5]:
import obspy
from obspy import UTCDateTime as utc
from obspy.clients.fdsn import Client
client = Client('IRIS')
import numpy as np
import os

### Data Description

The seismic data is downloaded from the IRIS/Earthscope Database using ObsPy. The data downloads as a trace, an array of data with attached metadata, which is then packaged into a stream, which can contain multiple traces. The daily seismic data records are then saved as mseed files, which preserve this data+metadata structure, but requires ObsPy or other specialized software to open.

In [6]:
#creating variables to download data
starttime = utc('2021-05-16T00:00:00')
endtime = starttime + 100 * (60*60*24)

#also add a buffer to both ends to chop off once the data has been filtered
#and downsampled, kind of arbitrary length, 5% of a day (default ObsPy taper length)
buffer = 60*60*24*0.05 #seconds

net = 'CC'
sta = 'SEP'
loc = '*' #wildcard, generally don't care about location code
cha = 'BHN' #horizontal component, as used in Zali et al

#create folder for numpy streams to go into and initialize filepath
!mkdir data
!mkdir data/raw
filepath = os.getcwd() + '/data/raw/'

#create arrays to save dates
dates = np.array([])

print(f'Downloading data from {starttime} to {endtime}')

#download the data piecemeal, here by day
for day in range(100):
    tr_length = 24*60*60

    #actually downloading
    st = client.get_waveforms(network=net,
                     station=sta,
                     location=loc,
                     channel=cha,
                     starttime=starttime-buffer,
                     endtime=starttime+buffer+tr_length)

    #instrument sampling rate (hz)
    freq = st[0].stats.sampling_rate

    #merge traces within stream, linearly interpolating any gaps
    st.merge(fill_value='interpolate')
    
    #generate filename, day number in front for convenience of reading in
    name = str(day+1)+'_sitkin.mseed'
    
    #save data as mseed, standard for storing seismic data. Preserves metadata and time series info
    st.write(filepath+name, format='MSEED')

    #adding date
    dates = np.append(dates, starttime.date)

    starttime += tr_length

#save dates list for future use
np.save(filepath+'date_list.csv', dates)

print(f'data download complete, saved to {filepath}')



mkdir: data: File exists
mkdir: data/raw: File exists
data download complete, saved to /Users/macbook/Documents/GitHub/MLGEO2024_Geldingadalir/notebooks/data/raw/


### Data Modalities and Formats

#### Data Modalities
The dataset consists of seismic data collected from a single horizontal component (BHN) of the AV.GSTD station located on the Great Sitkin Volcano. The data is recorded continuously over a period of 100 days, capturing the seismic activity before, during, and after the volcanic eruption.

#### Data Formats
1. **MSEED (Mini-SEED) Files**:
    - The seismic data is stored in Mini-SEED format, which is a compact binary format used for storing time series data. Each file contains a day's worth of seismic data, including metadata such as the sampling rate and station information.
    - Example file name: `1_sitkin.mseed`, `2_sitkin.mseed`, ..., `100_sitkin.mseed`.
2. **Numpy Arrays**:
    - The dates corresponding to each day's seismic data are stored in a numpy array and saved as a CSV file (`date_list.csv`). This array helps in mapping the MSEED files to their respective dates.
    - Example: `dates = np.array([datetime.date(2021, 5, 16), datetime.date(2021, 5, 17), ...])`.

#### Data Processing
- The raw seismic data is downloaded using the ObsPy library from the IRIS/Earthscope Database.
- The data is merged and interpolated to fill any gaps, ensuring a continuous time series.
- The processed data is saved in the MSEED format, preserving both the time series and metadata.

This structured approach ensures that the seismic data is well-organized and easily accessible for further analysis and processing.

In [ ]:
# display some data from the mseed file
print(f'Example data from {name}')
print(st[0].stats)
print()

# display the numpy array
print(f'Example data from {name}')
print(st[0].data)
print()

Example data from 2_sitkin.mseed
               network: AV
               station: GSTD
              location: 
               channel: BHN
             starttime: 2021-05-16T22:48:00.000000Z
               endtime: 2021-05-18T01:12:00.000000Z
         sampling_rate: 50.0
                 delta: 0.02
                  npts: 4752001
                 calib: 1.0
_fdsnws_dataselect_url: http://service.iris.edu/fdsnws/dataselect/1/query
               _format: MSEED
                 mseed: AttribDict({'dataquality': 'M', 'number_of_records': 11594, 'encoding': 'STEIM1', 'byteorder': '>', 'record_length': 512, 'filesize': 5936128})
            processing: ['ObsPy 1.4.1: trim(endtime=UTCDateTime(2021, 5, 18, 1, 12)::fill_value=None::nearest_sample=True::pad=False::starttime=UTCDateTime(2021, 5, 16, 22, 48))']

Example data from 2_sitkin.mseed
[134 137 141 ... 262 259 259]



### Calculate Distance Between Sensor and Volcano using Obspy

In [7]:
#Input Volcano Coords (degrees, north and east positive)
v_lat = 52.0765; v_lon = -176.1109 #Great Sitkin Volcano

In [8]:
#download inventory object of channels that fit specifiers set above, coordinates are contained here
inv = client.get_stations(network=net, station=sta, location=loc, channel=cha, starttime=starttime, 
                    endtime=endtime, level='channel')

s_lat = inv[0][0][0].latitude; s_lon = inv[0][0][0].longitude

In [9]:
#use obspy function to calculate great circle distance
distance, _, _ = obspy.geodetics.gps2dist_azimuth(v_lat, v_lon, s_lat, s_lon)
distance /= 1000 #convert from meters to km
print('Volcano to sensor distance: '+str(round(distance, 3))+' km')

Volcano to sensor distance: 3895.33 km
